In [1]:
import csv
import pandas as pd
import numpy as np
import networkx as nx
import sympy
import math
import matplotlib.pyplot as plt 
import knowledge_representation as KR
import itertools
import re

generating the knowledge graph that we previously built, to implement our applications on

In [2]:
G = KR.generate_graph(KR.import_equations(), KR.import_variables())
print([x for x in G.nodes()])
print([(x,y,z) for x,y,z in G.edges(data = True)])

['A', 'a', 'A_vec', 'alpha', 'B', 'beta', 'Bx', 'By', 'Bz', 'c', 'C', 'chi', 'd', 'D', 'd1', 'd2', 'delta', 'el_ct', 'E_n', 'E_den', 'Ef', 'epsilon', 'f', 'F', 'flux', 'foc', 'G', 'g', 'g_', 'gamma', 'H', 'h', 'hbar', 'H_G', 'I', 'I_0', 'I1', 'I2', 'Int', 'Int_0', 'j', 'Jz', 'K', 'k', 'k_f', 'k_G', 'k_spring', 'kappa', 'kb', 'L', 'L_rad', 'lambd', 'm', 'M', 'm_0', 'm1', 'm2', 'mob', 'mom', 'mu', 'mu_drift', 'Nn', 'n', 'n_rho', 'n_0', 'omega', 'omega_0', 'p', 'Pol', 'p_d', 'Pwr', 'pr', 'prob', 'q', 'q1', 'q2', 'r', 'R', 'r1', 'r2', 'rho', 'rho_c', 'rho_c_0', 'rho_0', 'sigma', 'sigma_den', 't', 'T', 't1', 'T1', 'T2', 'tau', 'theta', 'theta1', 'theta2', 'U', 'u', 'v', 'V', 'v1', 'V1', 'V2', 'Volt', 'w', 'x', 'x1', 'x2', 'x3', 'y', 'Y', 'y1', 'y2', 'y3', 'z', 'z1', 'z2', 'mu_S', 'L_ind', 'Z_1', 'Z_2', 'Length', 'Mass', 'Temperature', 'Voltage', 'Τime', (-3.0, 0.0, 0.0, 0.0, 0.0), (-3.0, 0.0, 1.0, 0.0, 0.0), (-2.0, 1.0, 0.0, 0.0, 1.0), (-2.0, 3.0, -1.0, 0.0, 2.0), (-2.0, 4.0, -1.0, 0.0, 2.0

Creating the list of formulas, which we use for testing

In [3]:
all_mysteries = KR.import_equations()

First perform a small scale experiment. we consider all dimensionless variables

In [4]:
print([(x,y,z) for x,y,z in G.edges(data = True) if z.get("IS_VARIABLE_IN") == True and x in
       [(x) for x,y,z in G.edges(data = True) if y== (0,0,0,0,0) and z.get("HAS_UNIT") == True]])

[('alpha', '((x1*(cos(omega*t)+alpha*cos(omega*t)**2)))', {'IS_VARIABLE_IN': True}), ('alpha', '((n*alpha/(1-(n*alpha/3))*epsilon*Ef))', {'IS_VARIABLE_IN': True}), ('alpha', '((1+n*alpha/(1-(n*alpha/3))))', {'IS_VARIABLE_IN': True}), ('alpha', '((mom*H/(kb*T)+(mom*alpha)/(epsilon*c**2*kb*T)*M))', {'IS_VARIABLE_IN': True}), ('alpha', '((2*pi*alpha/(n*d)))', {'IS_VARIABLE_IN': True}), ('alpha', '((beta*(1+alpha*cos(theta))))', {'IS_VARIABLE_IN': True}), ('beta', '((beta*(1+alpha*cos(theta))))', {'IS_VARIABLE_IN': True}), ('chi', '((sigma_den/epsilon*1/(1+chi)))', {'IS_VARIABLE_IN': True}), ('chi', '((mom*(1+chi)*B))', {'IS_VARIABLE_IN': True}), ('delta', '((I1+I2+2*sqrt(I1*I2)*cos(delta)))', {'IS_VARIABLE_IN': True}), ('g_', '((g_*q*B/(2*m)))', {'IS_VARIABLE_IN': True}), ('g_', '((g_*mom*B*Jz/(h/(2*pi))))', {'IS_VARIABLE_IN': True}), ('gamma', '((1/(gamma-1)*pr*V))', {'IS_VARIABLE_IN': True}), ('gamma', '((1/(gamma-1)*kb*v/A))', {'IS_VARIABLE_IN': True}), ('gamma', '((sqrt(gamma*pr/rho))

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0:1], 'GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass


In [6]:
def generate_differentials_at_point(point, eqn):
    x = tf.Variable(point, dtype = float)  # Create a Tensorflow variable initialized to 1.0

    with tf.GradientTape() as t3: 
        with tf.GradientTape() as t2:
            with tf.GradientTape() as t1:
                y = eval(eqn)

      # Compute the gradient inside the outer `t2` context manager
      # which means the gradient computation is differentiable as well.
            dy_dx = t1.gradient(y, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
        d2y_dx2 = t2.gradient(dy_dx, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
    d3y_dx3 = t3.gradient(d2y_dx2, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
    return [y.numpy(), dy_dx.numpy(), d2y_dx2.numpy(), d3y_dx3.numpy()]

def generate_taylor_series(arr):
    series = []
    for i in range(len(arr)):
        series.append(arr[i]/math.factorial(i))
    return series

# print((generate_differentials_at_point(1, tf.math.sin)))
# print(generate_taylor_series(generate_differentials_at_point(1, tf.math.sin)))

In [7]:
import automatic_differentiation as AD

def local_power(y):
    def pow(x):
        return tf.math.pow(x,y)
    return pow

point = 1

list_of_equations = ["tf.math.sin(x)", "tf.math.cos(x)", "tf.math.log(x)", "tf.math.exp(x)", "(x**2)",
                     "tf.math.sqrt(x)", "x**3", "tf.math.sinh(x)", "tf.math.cosh(x)", "tf.math.reciprocal(x)"]

list_compositionality = ["tf.math.sin(", "tf.math.cos(", "tf.math.log(", "tf.math.exp(", 
                     "tf.math.sqrt(", "tf.math.sinh(", "tf.math.cosh(", "tf.math.reciprocal("]

all_possible_equations = []

for i in list_compositionality:
    for j in list_of_equations:
        all_possible_equations.append(i+j+")")
        
for i in list_of_equations:
    all_possible_equations.append("(" + i + ")**3")
    all_possible_equations.append("(" + i + ")**2")


for i in list_of_equations:
    for j in list_of_equations:
        if i != j:
            all_possible_equations.append(i+"*"+j)
            all_possible_equations.append(i+"+"+j)
        all_possible_equations.append(i+"-"+j)
        all_possible_equations.append(i+"/"+j)

print(all_possible_equations)

taylor_series_ground_truth = []

for i in all_possible_equations:
    print(i)
    taylor_series_ground_truth.append(generate_taylor_series(generate_differentials_at_point(point, i)))
    
print(taylor_series_ground_truth)

def maape(actual, pred):
    return np.mean(np.arctan(np.abs((np.array(actual) - np.array(pred)) / np.array(pred)))) * 100

from sklearn.metrics import mean_squared_error
    

['tf.math.sin(tf.math.sin(x))', 'tf.math.sin(tf.math.cos(x))', 'tf.math.sin(tf.math.log(x))', 'tf.math.sin(tf.math.exp(x))', 'tf.math.sin((x**2))', 'tf.math.sin(tf.math.sqrt(x))', 'tf.math.sin(x**3)', 'tf.math.sin(tf.math.sinh(x))', 'tf.math.sin(tf.math.cosh(x))', 'tf.math.sin(tf.math.reciprocal(x))', 'tf.math.cos(tf.math.sin(x))', 'tf.math.cos(tf.math.cos(x))', 'tf.math.cos(tf.math.log(x))', 'tf.math.cos(tf.math.exp(x))', 'tf.math.cos((x**2))', 'tf.math.cos(tf.math.sqrt(x))', 'tf.math.cos(x**3)', 'tf.math.cos(tf.math.sinh(x))', 'tf.math.cos(tf.math.cosh(x))', 'tf.math.cos(tf.math.reciprocal(x))', 'tf.math.log(tf.math.sin(x))', 'tf.math.log(tf.math.cos(x))', 'tf.math.log(tf.math.log(x))', 'tf.math.log(tf.math.exp(x))', 'tf.math.log((x**2))', 'tf.math.log(tf.math.sqrt(x))', 'tf.math.log(x**3)', 'tf.math.log(tf.math.sinh(x))', 'tf.math.log(tf.math.cosh(x))', 'tf.math.log(tf.math.reciprocal(x))', 'tf.math.exp(tf.math.sin(x))', 'tf.math.exp(tf.math.cos(x))', 'tf.math.exp(tf.math.log(x))', 

tf.math.sin(tf.math.cos(x))
tf.math.sin(tf.math.log(x))
tf.math.sin(tf.math.exp(x))
tf.math.sin((x**2))
tf.math.sin(tf.math.sqrt(x))
tf.math.sin(x**3)
tf.math.sin(tf.math.sinh(x))
tf.math.sin(tf.math.cosh(x))
tf.math.sin(tf.math.reciprocal(x))
tf.math.cos(tf.math.sin(x))
tf.math.cos(tf.math.cos(x))
tf.math.cos(tf.math.log(x))
tf.math.cos(tf.math.exp(x))
tf.math.cos((x**2))
tf.math.cos(tf.math.sqrt(x))
tf.math.cos(x**3)
tf.math.cos(tf.math.sinh(x))
tf.math.cos(tf.math.cosh(x))
tf.math.cos(tf.math.reciprocal(x))
tf.math.log(tf.math.sin(x))
tf.math.log(tf.math.cos(x))
tf.math.log(tf.math.log(x))
tf.math.log(tf.math.exp(x))
tf.math.log((x**2))
tf.math.log(tf.math.sqrt(x))
tf.math.log(x**3)
tf.math.log(tf.math.sinh(x))
tf.math.log(tf.math.cosh(x))
tf.math.log(tf.math.reciprocal(x))
tf.math.exp(tf.math.sin(x))
tf.math.exp(tf.math.cos(x))
tf.math.exp(tf.math.log(x))
tf.math.exp(tf.math.exp(x))
tf.math.exp((x**2))
tf.math.exp(tf.math.sqrt(x))
tf.math.exp(x**3)
tf.math.exp(tf.math.sinh(x))
tf.m

tf.math.sqrt(x)-tf.math.sinh(x)
tf.math.sqrt(x)/tf.math.sinh(x)
tf.math.sqrt(x)*tf.math.cosh(x)
tf.math.sqrt(x)+tf.math.cosh(x)
tf.math.sqrt(x)-tf.math.cosh(x)
tf.math.sqrt(x)/tf.math.cosh(x)
tf.math.sqrt(x)*tf.math.reciprocal(x)
tf.math.sqrt(x)+tf.math.reciprocal(x)
tf.math.sqrt(x)-tf.math.reciprocal(x)
tf.math.sqrt(x)/tf.math.reciprocal(x)
x**3*tf.math.sin(x)
x**3+tf.math.sin(x)
x**3-tf.math.sin(x)
x**3/tf.math.sin(x)
x**3*tf.math.cos(x)
x**3+tf.math.cos(x)
x**3-tf.math.cos(x)
x**3/tf.math.cos(x)
x**3*tf.math.log(x)
x**3+tf.math.log(x)
x**3-tf.math.log(x)
x**3/tf.math.log(x)
x**3*tf.math.exp(x)
x**3+tf.math.exp(x)
x**3-tf.math.exp(x)
x**3/tf.math.exp(x)
x**3*(x**2)
x**3+(x**2)
x**3-(x**2)
x**3/(x**2)
x**3*tf.math.sqrt(x)
x**3+tf.math.sqrt(x)
x**3-tf.math.sqrt(x)
x**3/tf.math.sqrt(x)
x**3-x**3
x**3/x**3
x**3*tf.math.sinh(x)
x**3+tf.math.sinh(x)
x**3-tf.math.sinh(x)
x**3/tf.math.sinh(x)
x**3*tf.math.cosh(x)
x**3+tf.math.cosh(x)
x**3-tf.math.cosh(x)
x**3/tf.math.cosh(x)
x**3*tf.math.rec

In [8]:
print(len(all_possible_equations))

480


In [9]:
# Test equations have the following taylor series:
# print(taylor_series_ground_truth[6])    
# print(taylor_series_ground_truth[16])    
# print(taylor_series_ground_truth[26])    
# print(taylor_series_ground_truth[36])    
# print(taylor_series_ground_truth[45])    
# print(taylor_series_ground_truth[56])    
# print(taylor_series_ground_truth[66])    
# print(taylor_series_ground_truth[76])    
# print(taylor_series_ground_truth[86])    
# print(taylor_series_ground_truth[96])    

list_of_equations_np = ["np.sin(x)", "np.cos(x)", "np.log(x)", "np.exp(x)", "(x**2)",
                     "np.sqrt(x)", "x**3", "np.sinh(x)", "np.cosh(x)", "np.reciprocal(x)"]   

list_compositionality = ["np.sin(", "np.cos(", "np.log(", "np.exp(", 
                     "np.sqrt(", "np.sinh(", "np.cosh(", "np.reciprocal("]

all_possible_equations_np = []

for i in list_compositionality:
    for j in list_of_equations:
        all_possible_equations_np.append(i+j+")")
        
for i in list_of_equations:
    all_possible_equations_np.append("(" + i + ")**3")
    all_possible_equations_np.append("(" + i + ")**2")

    
for i in list_of_equations_np:
    for j in list_of_equations_np:
        if i != j:
            all_possible_equations_np.append(i+"*"+j)
            all_possible_equations_np.append(i+"+"+j)
        all_possible_equations_np.append(i+"-"+j)
        all_possible_equations_np.append(i+"/"+j)

In [ ]:
total_results_mse = []
total_results_maape = []

for test in range(500):
    
    try:
        x = np.random.rand(10000)*2+1
        y = eval(all_possible_equations_np[test])


        results = []

        for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
            results.append([np.array(AD.multiple_gradients(x, y, point = point, activation = i, learning_rate = 0.01)).reshape(4)])
        results.append(np.array(AD.three_grads(x, y, point = point)).reshape(4))   
        
        activation_rank_mse = []
        activation_rank_maape = []
        for i in results:
            print("predicting using function")
            mse_results = []
            maape_results = []
            for j in taylor_series_ground_truth:
                try:
                    mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
                except:
                    mse_results.append(100000000)
                try:
                    maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
                except:
                    maape_results.append(100000000)
            rank_mse = np.array(mse_results).argsort().argsort()[test]+1
            rank_maape = np.array(maape_results).argsort().argsort()[test]+1
            print("suggests the following with mse: ", rank_mse)
            print("suggests the following with maape: ", rank_maape)
            activation_rank_mse.append(rank_mse)
            activation_rank_maape.append(rank_maape)
        total_results_mse.append(activation_rank_mse)
        total_results_maape.append(activation_rank_maape)
        
    except:
        total_results_mse.append([100000000,100000000,100000000,100000000,100000000])
        total_results_maape.append([100000000,100000000,100000000,100000000,100000000])   

In [11]:
print(total_results_mse)
print(total_results_maape)

[[28, 28, 37, 28, 23], [44, 44, 6, 47, 56], [117, 114, 33, 115, 69], [319, 319, 323, 319, 319], [276, 271, 288, 272, 282], [13, 13, 2, 13, 11], [421, 421, 421, 421, 421], [142, 144, 167, 146, 133], [123, 117, 96, 122, 94], [45, 48, 27, 47, 36], [24, 27, 47, 24, 39], [28, 27, 30, 28, 21], [46, 57, 44, 47, 39], [376, 378, 375, 380, 373], [242, 243, 255, 242, 242], [20, 20, 1, 20, 5], [384, 387, 387, 384, 384], [150, 152, 163, 153, 149], [94, 94, 112, 95, 82], [191, 190, 175, 191, 188], [32, 34, 105, 32, 59], [307, 307, 307, 307, 307], [474, 470, 471, 467, 476], [61, 64, 1, 70, 70], [259, 254, 119, 259, 230], [31, 23, 13, 31, 17], [331, 314, 177, 335, 315], [167, 153, 10, 168, 129], [84, 71, 1, 83, 56], [42, 42, 18, 43, 40], [57, 88, 25, 64, 26], [177, 165, 33, 177, 126], [70, 63, 6, 62, 56], [16, 355, 16, 16, 355], [48, 44, 431, 44, 354], [1, 1, 3, 1, 4], [449, 352, 62, 62, 352], [28, 28, 454, 28, 351], [6, 6, 22, 6, 350], [416, 416, 414, 416, 416], [16, 19, 17, 16, 14], [347, 347, 347, 

In [12]:
np.savetxt('embeddings_twostep_onedim_mse_compositionality.csv', total_results_mse, delimiter=',', fmt = '%s') 
np.savetxt('embeddings_twostep_onedim_maape_compositionality.csv', total_results_maape, delimiter=',', fmt = '%s') 